## Setting up Orchestrator Agent A2A

In the previous module, we've launched two agents, using AgentCore Runtime, that supports A2A invocations.

In this lab, we're going to add an orchestrator, that will invoke our sub-agents.

<img src="images/architecture.png" style="width: 80%;">

So let's get started!

### Setup

Import required dependencies

In [ ]:
# Import libraries
import os
import json
import requests
import boto3
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()
region = boto_session.region_name

Retrieve information from previous LABs, so we can use it during this one.

In [ ]:
%store -r

### 1 - Create Code for the orchestrator agent

Let's generate Python code that will be used for our orchestrator, and lately will be deployed in AgentCore.

In [ ]:
%%writefile agents/orchestrator.py
import logging
import json
import base64
import hmac
import hashlib

from boto3.session import Session
from uuid import uuid4
from urllib.parse import quote

import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart

from helpers.utils import get_cognito_secret, reauthenticate_user, get_ssm_parameter, SSM_DOCS_AGENT_ARN, SSM_BLOGS_AGENT_ARN

from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from fastapi import HTTPException


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
DEFAULT_TIMEOUT = 300  # set request timeout to 5 minutes
username = "testuser"
MCP_AGENT_ARN = get_ssm_parameter(SSM_DOCS_AGENT_ARN)
BLOG_AGENT_ARN = get_ssm_parameter(SSM_BLOGS_AGENT_ARN)
boto_session = Session()
region = boto_session.region_name


app = BedrockAgentCoreApp()


def create_message(*, role: Role = Role.user, text: str) -> Message:
    return Message(
        kind="message",
        role=role,
        parts=[Part(TextPart(kind="text", text=text))],
        message_id=uuid4().hex,
    )

async def send_sync_message(message: str, agent_arn: str):
    escaped_agent_arn = quote(agent_arn, safe='')
    runtime_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/"
    
    session_id = str(uuid4())
    secret = json.loads(get_cognito_secret())
    bearer_token = reauthenticate_user(secret.get("client_id"), secret.get("client_secret"))
    
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id
    }
        
    async with httpx.AsyncClient(timeout=DEFAULT_TIMEOUT, headers=headers) as httpx_client:
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=runtime_url)
        agent_card = await resolver.get_agent_card()

        config = ClientConfig(httpx_client=httpx_client, streaming=False)
        factory = ClientFactory(config)
        client = factory.create(agent_card)

        msg = create_message(text=message)

        async for event in client.send_message(msg):
            if isinstance(event, Message):
                return event
            elif isinstance(event, tuple) and len(event) == 2:
                return event[0]
            else:
                return event

@tool
async def send_mcp_message(message: str):
    return await send_sync_message(message, MCP_AGENT_ARN)


@tool
async def send_blog_message(message: str):
    return await send_sync_message(message, BLOG_AGENT_ARN)


system_prompt = """You are an orchestrator, that will invoke other agents to get information about:

- AWS Documentation: Most recent data on AWS docs, from AWS Docs MCP
- AWS Blogs and News: Agent that query in the web for latest AWS Blogs and news

Key capabilities:
- Search and retrieve information from AWS service documentation on AWS Documentation Tool
- Search and retrieve information from AWS Blogs and News if asked for a specific subject

Considerations:

- all your queries should be summarized and optimized
- ask underlying agents/tools to summarize and shorten answers
- If you're unsure about something, clearly state your limitations
- Try to simplify/summarize answers to make it faster, small and objective

"""

agent = Agent(system_prompt=system_prompt, 
              tools=[send_mcp_message, send_blog_message],
              name="AWS Orchestration Agent",
              description="An agent to orchestrate sub-agents")

@app.entrypoint
async def invoke_agent(payload, context):
    logger.info("Received invocation request")

    logger.info(f"Payload: {payload}")
    logger.info(f"Context: {context}")
    try:

        # Extract user prompt
        user_prompt = payload.get("prompt", "")
        if not user_prompt:
            raise HTTPException(
                status_code=400,
                detail="No prompt found in input. Please provide a 'prompt' key in the input.",
            )
        
        session_id = context.session_id
        
        logger.info(f"Processing query: {user_prompt}")

        # Get the agent stream
        agent_stream = agent.stream_async(user_prompt)

        async for event in agent_stream:
            yield event

    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"Agent processing failed: {e}")
        logger.exception("Full exception details:")
        raise HTTPException(
            status_code=500, detail=f"Agent processing failed: {str(e)}"
        )

if __name__ == "__main__":
    app.run()

#### 1.1 - Create IAM Role for the Agent

In [ ]:
from helpers.utils import create_agentcore_runtime_execution_role, ORCHESTRATOR_ROLE_NAME

agent_name="aws_orchestrator_assistant"

execution_role_arn = create_agentcore_runtime_execution_role(ORCHESTRATOR_ROLE_NAME)

### 2 - Deploy to AgentCore Runtime

Now, let's deploy the orchestrator in the AgentCore Runtime.

Note that in this example, we're not adding `protocol` parameter. Which means that this will be a HTTP agent.

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime = Runtime()

# Configure the deployment
response = agentcore_runtime.configure(
    entrypoint="agents/orchestrator.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [COGNITO_CLIENT_ID],
            "discoveryUrl": DISCOVERY_URL,
        }
    },
)

print("Configuration completed:", response)

In [ ]:
launch_result = agentcore_runtime.launch()
print("Launch completed:", launch_result.agent_arn)

agent_arn = launch_result.agent_arn

**Check Deployment Status**

Let's check if deployment is completed:

In [ ]:
import time

# Wait for the agent to be ready
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"Waiting for deployment... Current status: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

print(f"Final status: {status}")

#### 2.1 - Export and save outputs

Export variables to be used in clean up notebook:

In [ ]:
ORCHESTRATION_ID = launch_result.agent_id
ORCHESTRATION_ARN = launch_result.agent_arn
ORCHESTRATION_NAME = agent_name

%store ORCHESTRATION_ID
%store ORCHESTRATION_ARN
%store ORCHESTRATION_NAME

### 3 - Invoking A2A agents using an orchestrator agent

Firstly, let's refresh the auth token:

In [ ]:
from helpers.utils import reauthenticate_user

bearer_token = reauthenticate_user(
    COGNITO_CLIENT_ID,
    COGNITO_SECRET
)

Now, let's invoke our orchestrator to check AWS Docs, making a call to our first agent, using A2A:

In [ ]:
import requests
import json
import uuid
from urllib.parse import quote

headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': str(uuid.uuid4())
}

prompt = {"prompt": "What is DynamoDB?"}

escaped_agent_arn = quote(ORCHESTRATION_ARN, safe='')

response = requests.post(
    f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations',
    headers=headers,
    data=json.dumps(prompt)
)

for line in response.iter_lines(decode_unicode=True):
    if line.startswith('data: '):
        data = line[6:]
        try:
            parsed = json.loads(data)
            print(parsed)
        except:
            print(data)

In [ ]:
import uuid

headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': str(uuid.uuid4())
}

prompt = {"prompt": "Give me the latest published blog for Bedrock AgentCore?"}

escaped_agent_arn = quote(ORCHESTRATION_ARN, safe='')

response = requests.post(
    f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations',
    headers=headers,
    data=json.dumps(prompt)
)

for line in response.iter_lines(decode_unicode=True):
    if line.startswith('data: '):
        data = line[6:]
        try:
            parsed = json.loads(data)
            print(parsed)
        except:
            print(data)

Congratulations, you have deployed the complete solution, using A2A protocol on Amazon AgentCore Runtime.